# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_df = pd.read_csv("./city_weather.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,New Norfolk,-42.7826,147.0587,51.71,80,100,3.00,AU,1661885990
1,1,Constitución,-35.3333,-72.4167,57.47,71,68,7.90,CL,1661885991
2,2,Cherskiy,68.7500,161.3000,34.66,89,23,12.48,RU,1661885991
3,3,Ushuaia,-54.8000,-68.3000,49.66,57,75,5.75,AR,1661885992
4,4,Baní,18.2833,-70.3333,90.84,55,99,12.30,DO,1661885992


In [8]:
weather_df.count()

Unnamed: 0    591
City          591
Lat           591
Lng           591
Max Temp      591
Humidity      591
Cloudiness    591
Wind Speed    591
Country       585
Date          591
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
heatmap_humidity = gmaps.figure(map_type="HYBRID")
max_int = weather_df["Humidity"].max()
heat_layer = gmaps.heatmap_layer(locations=weather_df[['Lat', 'Lng']], weights=weather_df['Humidity'].astype(float), dissipating=False,
    max_intensity=max_int, point_radius=1)
heatmap_humidity.add_layer(heat_layer)
heatmap_humidity
plt.savefig("./Images/Vacation-heatmap_humidity.png")

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Weather conditions:
# Max Temp: between 65 and 70 degrees
# Wind Speed: 10 mph
# Cloudiness: 40-50%
perfect_weather = weather_df.loc[(weather_df["Max Temp"] >=60)&
    (weather_df["Max Temp"]<=70)&
    (weather_df["Wind Speed"]<=10)&
    (weather_df["Cloudiness"]<=50)&
    (weather_df["Cloudiness"]>=20), :]
perfect_weather.reset_index(drop=True, inplace=True)
perfect_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,23,Pisco,-13.7000,-76.2167,64.45,68,40,9.22,PE,1661885702
1,264,Ampanihy,-24.7000,44.7500,65.62,81,47,8.79,MG,1661886096
2,265,Shebunino,46.4339,141.8603,62.74,91,24,4.23,RU,1661886097
3,355,Rohru,31.2167,77.7500,63.81,95,22,1.63,IN,1661886129
4,461,Manakara,-22.1333,48.0167,68.38,96,31,5.70,MG,1661886168


In [11]:
perfect_weather.count()

Unnamed: 0    9
City          9
Lat           9
Lng           9
Max Temp      9
Humidity      9
Cloudiness    9
Wind Speed    9
Country       9
Date          9
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = perfect_weather.copy()
hotel_df["Hotel Name"]=""
hotel_df.columns

Index(['Unnamed: 0', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date', 'Hotel Name'],
      dtype='object')

In [13]:
for r in range(0, len(hotel_df)):
    h_lat = hotel_df["Lat"][r]
    h_lng = hotel_df["Lng"][r]
    hotel_coord = f"{h_lat},{h_lng}"
    params = {
        "location": hotel_coord,
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
    }
    response=requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    hotel_df["Hotel Name"][r] = response["results"][0]["name"]
hotel_df.head()

C:\Users\krist\AppData\Local\Temp\ipykernel_13012\1839956463.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"][r] = response["results"][0]["name"]


IndexError: list index out of range

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations)
heatmap_humidity.add_layer(markers)
# Display figure
plt.savefig("./Images/Vacation-heatmap_humitiy_marker.png")
plt.show()

<Figure size 432x288 with 0 Axes>